# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('data/python.db')

# Read 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql_query("SELECT * FROM school", con)
student_data = pd.read_sql_query("SELECT * FROM student", con)

In [2]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Total number of schools

schools = school_data_complete['school_name'].nunique()
schools

15

In [4]:
# Total number of students

students = school_data_complete['Student ID'].nunique()
students

39170

In [5]:
# Total budget

budget = sum(school_data_complete['budget'].unique())
budget

24649428

In [6]:
# Average math score

avg_math = school_data_complete['math_score'].mean()
avg_math

78.98537145774827

In [7]:
# Average reading score

avg_reading = school_data_complete['reading_score'].mean()
avg_reading

81.87784018381414

In [8]:
# Overall average score

ovr_avg = (avg_math + avg_reading)/2
ovr_avg

80.43160582078121

In [9]:
# Percentage of passing math and reading (70 or greater)

passing_math = (school_data_complete[school_data_complete['math_score'] >= 70].shape[0]/students)*100
passing_math

passing_reading = (school_data_complete[school_data_complete['reading_score'] >= 70].shape[0]/students)*100
passing_reading

85.80546336482001

In [10]:
# Create a District Summary
district_summary = pd.DataFrame({
    'Total Number of Schools': [schools],
    'Total Number of Students': [students],
    'Total Budget': [budget],
    'Average Math Score': [avg_math],
    'Average Reading Score': [avg_reading],
    'Overall Average Score': [ovr_avg],
    'Percentage Passing Math': [passing_math],
    'Percentage Passing Reading': [passing_reading]})

formatted_district_summary = district_summary.style.format({
    'Total Budget': '${:,.2f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    'Overall Average Score': '{:.2f}',
    'Percentage Passing Math': '{:.2f}%',
    'Percentage Passing Reading': '{:.2f}%',
    'Total Number of Students': '{:,}'})

formatted_district_summary

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,Percentage Passing Math,Percentage Passing Reading
0,15,"39,170","$24,649,428.00",78.99,81.88,80.43,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [11]:
#  Sort and display the top five schools in overall passing rate

school_summary = school_data_complete.groupby('school_name').agg({
    'Student ID': 'count',
    'math_score': 'mean',
    'reading_score': 'mean',
    'budget': 'first'
}).rename(columns={
    'Student ID': 'Total Number of Students', 
    'math_score': 'Average Math Score', 
    'reading_score': 'Average Reading Score',
    'budget': 'Budget'
})

school_summary['Percent Passing Math'] = (school_data_complete[school_data_complete['math_score'] >= 70]
                                    .groupby('school_name')['Student ID']
                                    .count() / school_summary['Total Number of Students'] * 100)

school_summary['Percent Passing Reading'] = (school_data_complete[school_data_complete['reading_score'] >= 70]
                                       .groupby('school_name')['Student ID']
                                       .count() / school_summary['Total Number of Students'] * 100)

school_summary['Overall Passing Rate'] = (school_summary['Percent Passing Math'] + school_summary['Percent Passing Reading']) / 2
top_five = school_summary.sort_values('Overall Passing Rate', ascending=False).head(5)

top_five

,Total Number of Students,Average Math Score,Average Reading Score,Budget,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
school_name,,,,,,,
Cabrera High School,1858,83.061895,83.975780,1081356,94.133477,97.039828,95.586652
Thomas High School,1635,83.418349,83.848930,1043130,93.272171,97.308869,95.290520
Pena High School,962,83.839917,84.044699,585858,94.594595,95.945946,95.270270
Griffin High School,1468,83.351499,83.816757,917500,93.392371,97.138965,95.265668
Wilson High School,2283,83.274201,83.989488,1319574,93.867718,96.539641,95.203679


In [12]:
# Calculate total school budget

total_budget_top_five = top_five['Budget'].sum()
total_budget_top_five

4947418

In [13]:
# Calculate per student budget

per_student_budget = total_budget_top_five/top_five['Total Number of Students'].sum()
per_student_budget

602.9025103582744

In [14]:
# Cacluate the avg math and reading score

top_five_avg_math = top_five['Average Math Score'].mean()
top_five_avg_reading = top_five['Average Reading Score'].mean()
print(top_five_avg_math, top_five_avg_reading)

83.38917184496529 83.93513072539257


In [15]:
top_five_metrics = pd.DataFrame({
    'Total Budget of Top Five Schools': [total_budget_top_five],
    'Per Student Budget': [per_student_budget],
    'Average Math Score of Top Five Schools': [top_five_avg_math],
    'Average Reading Score of Top Five Schools': [top_five_avg_reading]})

top_five_metrics

,Total Budget of Top Five Schools,Per Student Budget,Average Math Score of Top Five Schools,Average Reading Score of Top Five Schools
0,4947418,602.90251,83.389172,83.935131


#### Find the passing rate for math and reading (above 70 points)

In [16]:
# Find the total counts of math result

count_of_math_scores = school_data_complete['math_score'].shape[0]
# Find the counts for math result in each school that pass 70 or higher

count_of_math_passing = school_data_complete[school_data_complete['math_score'] >= 70].shape[0]
# Calculate the math passing rate

math_passing = (count_of_math_passing/count_of_math_scores) * 100
math_passing


74.9808526933878

In [17]:
# Find the total counts of read result

count_of_reading_scores = school_data_complete['reading_score'].shape[0]
# Find the counts for read result in each school that pass 70 or higher

count_of_reading_passing = school_data_complete[school_data_complete['reading_score'] >= 70].shape[0]
# Calculate the read passing rate

reading_passing = (count_of_reading_passing/count_of_reading_scores) * 100
reading_passing

85.80546336482001

In [18]:
# Calculate the overall passing rate (average of the math and reading passing rate)

passing_ovr = (reading_passing+math_passing)/2
passing_ovr

80.39315802910392

### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [19]:
#  Sort and display the worst five schools in overall passing rate

bottom_five = school_summary.sort_values('Overall Passing Rate', ascending=False).tail(5)
bottom_five

,Total Number of Students,Average Math Score,Average Reading Score,Budget,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
school_name,,,,,,,
Ford High School,2739,77.102592,80.746258,1763916,68.309602,79.299014,73.804308
Johnson High School,4761,77.072464,80.966394,3094650,66.057551,81.222432,73.639992
Huang High School,2917,76.629414,81.182722,1910635,65.683922,81.316421,73.500171
Figueroa High School,2949,76.711767,81.158020,1884411,65.988471,80.739234,73.363852
Rodriguez High School,3999,76.842711,80.744686,2547363,66.366592,80.220055,73.293323


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [20]:
# Create table that lists the average math score for each school of each grade level.

math_scores_by_grade = pd.DataFrame()

In [26]:
# Calculate the average math score for 9th grade in each school

ninth = school_data_complete[school_data_complete['grade']=='9th'].groupby('school_name')['math_score'].mean()
ninth_df = ninth.reset_index()
ninth_df

,school_name,math_score
0,Bailey High School,77.083676
1,Cabrera High School,83.094697
2,Figueroa High School,76.403037
3,Ford High School,77.361345
4,Griffin High School,82.044010
5,Hernandez High School,77.438495
6,Holden High School,83.787402
7,Huang High School,77.027251
8,Johnson High School,77.187857
9,Pena High School,83.625455


In [27]:
# Calculate the average math score for 10th grade in each school

tenth = school_data_complete[school_data_complete['grade']=='10th'].groupby('school_name')['math_score'].mean()
tenth_df = tenth.reset_index()
tenth_df

,school_name,math_score
0,Bailey High School,76.996772
1,Cabrera High School,83.154506
2,Figueroa High School,76.539974
3,Ford High School,77.672316
4,Griffin High School,84.229064
5,Hernandez High School,77.337408
6,Holden High School,83.429825
7,Huang High School,75.908735
8,Johnson High School,76.691117
9,Pena High School,83.372000


In [28]:
# Calculate the average math score for 11th grade in each school

eleventh = school_data_complete[school_data_complete['grade']=='11th'].groupby('school_name')['math_score'].mean()
eleventh_df = eleventh.reset_index()
eleventh_df

,school_name,math_score
0,Bailey High School,77.515588
1,Cabrera High School,82.765560
2,Figueroa High School,76.884344
3,Ford High School,76.918058
4,Griffin High School,83.842105
5,Hernandez High School,77.136029
6,Holden High School,85.000000
7,Huang High School,76.446602
8,Johnson High School,77.491653
9,Pena High School,84.328125


In [33]:
# Calculate the average math score for 12th grade in each school

twelfth = school_data_complete[school_data_complete['grade']=='12th'].groupby('school_name')['math_score'].mean()
twelfth_df = twelfth.reset_index()
twelfth_df

merge =  pd.merge(eleventh_df, twelfth_df, on='school_name', suffixes=('_11th_grade', '_12th_grade'))
merge2 = pd.merge(ninth_df, tenth_df, on ='school_name', suffixes=('_9th_grade','_10th_grade'))

math_scores_by_grade = pd.merge(merge, merge2, on='school_name')

rename_columns = ['School Name',
                  '9th Average Math Score',
                  
math_scores_by_grade.columns = [

Index(['school_name', 'math_score_11th_grade', 'math_score_12th_grade',
       'math_score_9th_grade', 'math_score_10th_grade'],
      dtype='object')

In [ ]:
# Create table that lists the average reading score for each school of each grade level.



In [ ]:
# Calculate the average reading score for 9th grade in each school




In [ ]:
# Calculate the average reading score for 10th grade in each school




In [ ]:
# Calculate the average reading score for 11th grade in each school




In [ ]:
# Calculate the average reading score for 12th grade in each school




## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [ ]:
# Create a new column to show budget per student in each row



In [ ]:
# Create a new column to define the spending ranges per student




In [ ]:
# Calculate the average math score within each spending range





In [ ]:
# Calculate the percentage passing rate for math in each spending range





In [ ]:
# Calculate the percentage passing rate for reading in each spending range




In [ ]:
# Calculate the percentage overall passing rate in each spending range




### Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Create a new column for the bin groups


Look for the total count of test scores that pass 70% or higher




In [ ]:
# math_pass_size




In [ ]:
# read_pass_size




In [ ]:
# Calculate the overall passing rate for different school size




### Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
# Create bins and groups, school type {'Charter', 'District'}




Find counts of the passing 70 or higher score for the both test


In [ ]:
# math pass size




In [ ]:
# reading pass size



In [ ]:
# Calculate the overall passing rate

